In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime
import os

In [2]:
def save_world_to_database(df_world):
    connection_string = "postgres:postgres@localhost:5432/corona_db"
    engine = create_engine(f'postgresql://{connection_string}')
    connection = engine.connect()
    connection.execute( '''TRUNCATE TABLE daily_stats_world''' )
    connection.close()
    df_world.to_sql(name='daily_stats_world', con=engine, if_exists='append', index=False)

In [3]:
# Start here to process the world data (only call save_world_to_database)
url_world = "static/world_data/df_world_all.csv"
df_world = pd.read_csv(url_world)
df_world = df_world.drop(['country_2'], axis=1)
df_world.head()
save_world_to_database(df_world)

In [4]:
# query_str = open('static/sql/test_world.sql')
query_str = open('static/sql/world_top_10.sql')
query_text = ""
for text in query_str:
    query_text = query_text + text

connection_string = "postgres:postgres@localhost:5432/corona_db"
engine = create_engine(f'postgresql://{connection_string}')    
print(query_text)
df_query = pd.read_sql_query(query_text, con=engine)

df_query


select  
	country, 
	max(date), 
	sum(conf_count)-sum(cured_count)-sum(dead_count) positive, 
	sum(cured_count) cured, 
	sum(dead_count) dead
    from daily_stats_world 
    where date = (select max(date ) from daily_stats_world) 
    group by country 
    order by positive desc 
	limit 10


,country,max,positive,cured,dead
0,China,2020-02-28,39745,36291,2788
1,South Korea,2020-02-28,2302,22,13
2,Italy,2020-02-28,821,46,21
3,Others,2020-02-28,694,10,1
4,Iran,2020-02-28,281,73,34
5,Japan,2020-02-28,202,22,4
6,Hong Kong,2020-02-28,62,30,2
7,United States of America,2020-02-28,55,7,0
8,Kuwait,2020-02-28,45,0,0
9,France,2020-02-28,44,11,2
